# Project 4
## Nick Shinn and Brian Lee

In [14]:
!pip install folium
import folium, requests
import pandas, numpy as np
from folium.plugins import HeatMapWithTime

# download and prepare data for use
arrest_table = pandas.read_csv("http://www.hcbravo.org/IntroDataSci/misc/BPD_Arrests.csv")

In [15]:
# Swap misnamed columns
arrest_table["race_new"] = arrest_table["sex"]
arrest_table["sex_new"] = arrest_table["race"]
arrest_table["race"] = arrest_table["race_new"]
arrest_table["sex"] = arrest_table["sex_new"]
arrest_table = arrest_table.drop('race_new', 1)
arrest_table = arrest_table.drop('sex_new', 1)

# Drop entries with null location
arrest_table = arrest_table[pandas.notnull(arrest_table["Location 1"])]

# Convert locations to Lat and Long
arrest_table["lat"], arrest_table["long"] = arrest_table["Location 1"].str.split(",").str
arrest_table["lat"] = arrest_table["lat"].str.replace("(", "").astype(float)
arrest_table["long"] = arrest_table["long"].str.replace(")", "").astype(float)

arrest_table.head()

,arrest,age,sex,race,arrestDate,arrestTime,arrestLocation,incidentOffense,incidentLocation,charge,chargeDescription,district,post,neighborhood,Location 1,lat,long
1,11127013.0,37,M,B,01/01/2011,00:01,2000 Wilkens Ave,79-Other,Wilkens Av & S Payson St,1 1425,Reckless Endangerment || Hand Gun Violation,SOUTHERN,934.0,Carrollton Ridge,"(39.2814026274, -76.6483635135)",39.281403,-76.648364
2,11126887.0,46,M,B,01/01/2011,00:01,2800 Mayfield Ave,Unknown Offense,NaN,NaN,Unknown Charge,NORTHEASTERN,415.0,Belair-Edison,"(39.3227699160, -76.5735750473)",39.322770,-76.573575
3,11126873.0,50,M,B,01/01/2011,00:04,2100 Ashburton St,79-Other,2100 Ashburton St,1 1106,Reg Firearm:Illegal Possession || Hgv,WESTERN,735.0,Panway/Braddish Avenue,"(39.3117196723, -76.6623546313)",39.311720,-76.662355
4,11126968.0,33,M,B,01/01/2011,00:05,4000 Wilsby Ave,Unknown Offense,1700 Aliceanna St,NaN,Unknown Charge,NORTHERN,525.0,Pen Lucy,"(39.3382885254, -76.6045667070)",39.338289,-76.604567
5,11127041.0,41,M,B,01/01/2011,00:05,2900 Spellman Rd,81-Recovered Property,2900 Spelman Rd,1 1425,Reckless Endangerment || Handgun Violation,SOUTHERN,924.0,Cherry Hill,"(39.2449886230, -76.6273582432)",39.244989,-76.627358


In [16]:
# Get random sample of 5000 entries from dataset
rand_sample = arrest_table.sample(n=5000, axis=0)

# copy sorted version of data table to optimize performance
rand_sample_copy = rand_sample.sort_values(by='arrestTime')

In [11]:
# Helper function to convert HH:MM time string into minutes
def time_int(time):
    h, m = time.split(':')
    return int(h) * 60 + int(m)

In [12]:
# Creates array that defines each hour of the day (converted into minutes)
times = np.arange(0, 1440, 60)

arr =[]
# Gather all incidents in order of arrest time
# Drops rows as they are added into the heatmap data
data = []
for time in times:
    time_slot = []
    for index, row in rand_sample_copy.iterrows():
        if abs(time_int(row.arrestTime)-time) < 60: 
            time_slot.append([float(row.lat), float(row.long)])
            rand_sample_copy.drop(index, inplace=True)
        arr.append(time_int(row.arrestTime))
    data.append(time_slot)        

In [17]:
# Create instance of folium map
m = folium.Map(location=[39.29, -76.61], zoom_start=12)

# Add heat map data to folium map
HeatMapWithTime(data,auto_play=True).add_to(m)
m

We took a random sample of 5000 crimes from the Baltimore area, and organized them by location and time-of-day. We use this data to generate a heat map of arrests in the region hour-by-hour in a 24-hour period. This map can be used to visualize the most at-risk neighborhoods for crime in the city, based on time of day, in hopes of shedding light on any correlation that may exist between the two variables (time and location) in terms of crime frequency.